In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [5]:
import os
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings

# 1) 환경변수 확인 (없으면 직접 문자열로 넣어도 됨)
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
INDEX_NAME = "realestate"   # 너 인덱스 이름에 맞게 수정

# 2) Pinecone 연결
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)

# 3) 임베딩 모델 (너가 업로드 때 사용한 모델과 동일해야 함)
emb = OpenAIEmbeddings(model="text-embedding-3-large")

def test_search(query: str, top_k: int = 5):
    qvec = emb.embed_query(query)
    res = index.query(
        vector=qvec,
        top_k=top_k,
        include_metadata=True
    )
    return res

# 4) 테스트 질문 (일단 하나만)
query = "확정일자 안 받으면 보증금 우선변제 못 받나?"
res = test_search(query, top_k=5)

# 5) 보기 좋게 출력
for i, m in enumerate(res["matches"], 1):
    score = m.get("score")
    md = m.get("metadata", {})
    print("="*60)
    print(f"[{i}] score: {score:.4f}")
    print("source:", md.get("source"))
    print("law_type:", md.get("law_type"), "priority:", md.get("priority"))
    print("article:", md.get("article"), "chunk_id:", md.get("chunk_id"))
    text_preview = md.get("text") or md.get("chunk_text") or ""
    print("text preview:", (text_preview[:250] + "...") if len(text_preview) > 250 else text_preview)

[1] score: 0.4793
source: 주택임대차보호법(법률)(제21065호)(20260102).docx
law_type: 주택임대차보호법 priority: 1
article: 제3조의2 chunk_id: 항③
text preview: 제1항에 따라 우선변제를 받을 임차인 및 보증금 중 일정액의 범위와 기준은
[2] score: 0.4617
source: 주택임대차보호법(법률)(제21065호)(20260102).docx
law_type: 주택임대차보호법 priority: 1
article: 제6조의2 chunk_id: 항⑥
text preview: 제5항에 따른 손해배상액은 거절 당시 당사자 간에 손해배상액의 예정에 관한 합의가 이루어지지 않는 한 다음 각 호의 금액 중 큰 금액으로 한다.
1. 갱신거절 당시 월차임(차임 외에 보증금이 있는 경우에는 그 보증금을
[3] score: 0.4514
source: 전세피해법률상담사례집_250102.pdf
law_type: 사례 priority: 50
article: 사례 1 chunk_id: case_1_1
text preview: . 임차인 A는 임대차계약을 체결하고 동일자로 주택을 인도받고 전입신고 및 확정일자
를 필하였는데 A가 입주하는 같은 날 매매로 소유자가 변경되었으며 또한 소유권이전과
동시에 같은 날 은행에 근저당권이 설정되었습니다. 임차인 A의 보증금은 어떻게 되나요?
2. 임차인 B는 보증금, 임대기간을 24개월까지로 하는 임대차계약을 체결하고 전입 및
확정일자를 받았습니다. 그런데 임대인이 전입후본 주택을 C 법인에게 매도하였는데 B는
소유권이 변경되는 ...
[4] score: 0.4449
source: 주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙(대법원규칙)(제02986호)(20210610).docx
law_type: 규칙 priority: 3
article: 제8조 chunk_id: 항②
text preview: 확정일자부여기관은 평일 16시 이후에 전자확정일자 부여의 신

In [6]:
def build_context(matches, max_docs=3):
    """
    Pinecone 검색 결과(matches)를
    LLM용 context 문자열로 변환
    """
    chunks = []

    for m in matches[:max_docs]:
        md = m["metadata"]
        chunk = f"""
[출처]
- 법종류: {md.get("law_type")}
- 조문: {md.get("article")} {md.get("chunk_id")}
- 파일: {md.get("source")}

[내용]
{md.get("text")}
"""
        chunks.append(chunk.strip())

    return "\n\n---\n\n".join(chunks)

In [7]:
RAG_PROMPT = """
너는 주택 임대차 전문 법률 AI다.

아래 [법령/사례] 내용만 근거로 사용해서 질문에 답해라.
추측하거나 일반 상식을 섞지 마라.
근거가 부족하면 '제공된 자료만으로는 단정하기 어렵다'고 답해라.

[법령/사례]
{context}

[질문]
{question}

[답변 작성 가이드]
1. 결론을 먼저 간단히 말할 것
2. 관련 조문을 인용하며 설명할 것
3. 필요한 경우 주의사항을 덧붙일 것
"""

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",  # 또는 쓰던 모델
    temperature=0
)

def answer_with_rag(question: str, search_result):
    context = build_context(search_result["matches"])

    prompt = RAG_PROMPT.format(
        context=context,
        question=question
    )

    response = llm.invoke(prompt)
    return response.content

In [9]:
query = "확정일자 안 받으면 보증금 우선변제 못 받나?"
search_res = test_search(query, top_k=5)

answer = answer_with_rag(query, search_res)
print(answer)

확정일자를 받지 않으면 보증금 우선변제를 받을 수 없습니다.

주택임대차보호법 제3조의2 항③에 따르면, 임차인이 우선변제를 받기 위해서는 대항력을 갖추어야 하며, 대항력을 갖추기 위해서는 전입신고와 확정일자를 받아야 합니다. 확정일자는 임차인이 해당 주택에 대한 권리를 주장할 수 있는 법적 근거가 되므로, 이를 받지 않으면 우선변제를 받을 수 있는 자격이 상실됩니다.

따라서, 임차인은 반드시 전입신고와 함께 확정일자를 받아야 보증금에 대한 우선변제를 받을 수 있다는 점에 유의해야 합니다.
